In [2]:
from faim_hcs.hcs.Experiment import Experiment
from faim_hcs.records.PlateRecord import PlateRecord
from faim_hcs.records.WellRecord import WellRecord
from faim_hcs.records.OrganoidRecord import OrganoidRecord

from glob import glob

import time
import os
from os.path import basename, dirname, join

from os.path import isdir, dirname, split, basename, splitext, exists
from os import listdir
from tqdm.notebook import tqdm

import re
import tifffile
import warnings

import pandas as pd
import numpy as np



from run_platymatch_NAR220527 import runPM, runAffine, runFFD


pd.set_option("display.max_rows", None)

## Load experiments and images

In [3]:
RX_name = "R1"
#must be relative to R0
R0_dir = '/tungstenfs/scratch/gliberal/Users/repinico/Microscopy/Analysis/20220528_GCPLEX_redo/20220507GCPLEX_R0/summary.csv'
RX_dir = '/tungstenfs/scratch/gliberal/Users/repinico/Microscopy/Analysis/20220528_GCPLEX_redo/20220507GCPLEX_R1/summary.csv'

segname = 'NUC_SEG3D_220523'
ovr_channel = "C01"

In [4]:
R0 = Experiment()
R0.load(R0_dir)

RX = Experiment()
RX.load(RX_dir)

names = ["R0", RX_name]
exps = [R0, RX]

zipd = zip(names, exps)
rounds = dict(zipd)

## Run PlatyMatch linking

In [6]:
R0.only_iterate_over_wells(False)
R0.reset_iterator()

for organoid in R0:
    affine_matches = None
    ffd_matches = None
    
    if organoid.well.plate.plate_id in ["day4p5"]: 
        #print("skipping", organoid.well.plate.plate_id)
        continue #skip these timepoints
    
    well_id = organoid.well.well_id
    plate_id = organoid.well.plate.plate_id
    
    #load R0 data
    R0_obj = organoid.organoid_id
    R0_id = int(R0_obj.rpartition("_")[2])
    
#     if plate_id != "day3p5": #limit to 1 organoid
#         continue
    
#     if well_id != "B04": #limit to 1 organoid
#         continue
    
#     if R0_id != 4: #limit to 1 organoid
#         continue
       
    
    #load organoid measurement data for filtering
    R0_df_ovr = organoid.well.get_measurement('regionprops_ovr_C01')
    R0_df_ovr = R0_df_ovr.set_index('organoid_id')
    R0_df_org = organoid.get_measurement('regionprops_org_C01')
    
    #load linking data
    link_org = organoid.well.get_measurement('linking_ovr_C01_'+RX_name+'toR0')
    link_org_dict = link_org.set_index('R0_label').T.to_dict('index')['RX_label'] #R0 is key, RX is value
    
    #do not link organoids that don't have a RX-R0 match
    if R0_id not in link_org_dict: #move to next organoid if no match
        continue
    
    #do not link border organoids
    if R0_df_ovr.loc[R0_obj,'flag_tile_border'] == True:
        #print("skipping", plate_id, well_id, R0_obj, "touches border")
        continue
    
    if R0_df_org["abs_min"][0] == 0:
        #print("skipping", plate_id, well_id, R0_obj, "min = 0")
        continue
    
    try:
        R0_df = organoid.get_measurement('regionprops_nuc_C01')
        R0_raw = organoid.get_raw_data(ovr_channel)
        R0_seg = organoid.get_segmentation(segname)
    except Exception as e:
        print(e) #usually exception is that no nuc were detected so csv is empty. in this case, skip organoid
        continue
    
    #load RX data for organoids that pass
    RX_id = link_org_dict[R0_id]
    RX_obj = "object_"+ str(RX_id)
    
    try:
        RX_df = RX.plates[plate_id].wells[well_id].organoids[RX_obj].get_measurement('regionprops_nuc_C01')
        RX_raw = RX.plates[plate_id].wells[well_id].organoids[RX_obj].get_raw_data(ovr_channel)
        RX_seg = RX.plates[plate_id].wells[well_id].organoids[RX_obj].get_segmentation(segname)
    except Exception as e:
        print(e) #usually exception is that no nuc were detected so csv is empty. in this case, skip organoid
        continue 

    #convert df's to PlatyMatch compatible numpy array
    # N x 5 (first column is ids, last column is size)
    R0_numpy = R0_df[['nuc_id', 'x_pos_vox', 'y_pos_vox', 'z_pos_vox', 'volume_pix']].to_numpy()
    RX_numpy = RX_df[['nuc_id', 'x_pos_vox', 'y_pos_vox', 'z_pos_vox', 'volume_pix']].to_numpy()
    #Divid by z voxel anisotropy so that coordinates match label image!
    R0_numpy[:,3] *= 1/3
    R0_numpy[:,4] *= 1/3
    RX_numpy[:,3] *= 1/3
    RX_numpy[:,4] *= 1/3
    
    #skip organoids that have less than 4 nuclei
    if (R0_numpy.shape[0] <= 4) | (RX_numpy.shape[0] <= 4): #move to next organoid if no match
        continue
        
    #run PlatyMatch
    warnings.filterwarnings("ignore")
    ransac_iterations=4000
    icp_iterations=50
    
    print("matching of", R0_obj, "and", RX_obj, "in", plate_id, well_id)
    
    try:
        #affine_matches, ffd_matches = runPM(RX_numpy, R0_numpy, ransac_iterations, icp_iterations, RX_raw, R0_raw, RX_seg, R0_seg, "savename")
        
        affine_matches, transform_matrix_combined, confidence = runAffine(RX_numpy, R0_numpy, ransac_iterations, icp_iterations, save_images=False)
        
        affine_matches = pd.DataFrame(affine_matches, columns=['R0_nuc_id', 'RX_nuc_id', 'confidence'])
        affine_matches['R0_organoid_id'] = R0_obj
        affine_matches['RX_organoid_id'] = RX_obj
        affine_matches['plate_id'] = plate_id
        affine_matches['well_id'] = well_id
        
        # Save measurement into the organoid directory.
        name = "linking_nuc_affine_" + names[1] + "to" + names[0]
        path = join(organoid.organoid_dir, name + ".csv")
        affine_matches.to_csv(path, index=False) #saves csv

        # Add the measurement to the faim-hcs datastructure and save.
        organoid.add_measurement(name, path)
        organoid.save() #updates json file
        
    except Exception as e:
        print(R0_obj, RX_obj, e)
        #print(R0_numpy.shape, RX_numpy.shape)
    
    try:
        ffd_matches = runFFD(RX_numpy, R0_numpy, ransac_iterations, icp_iterations, RX_raw, R0_raw, RX_seg, R0_seg, "savename", transform_matrix_combined, confidence, save_images=False)
        
        ffd_matches = pd.DataFrame(ffd_matches, columns=['R0_nuc_id', 'RX_nuc_id'])
        #ffd_matches = pd.DataFrame(ffd_matches, columns=['R0_nuc_id', 'RX_nuc_id', 'confidence'])
        ffd_matches['R0_organoid_id'] = R0_obj
        ffd_matches['RX_organoid_id'] = RX_obj
        ffd_matches['plate_id'] = plate_id
        ffd_matches['well_id'] = well_id
        
        # Save measurement into the organoid directory.
        name = "linking_nuc_ffd_" + names[1] + "to" + names[0]
        path = join(organoid.organoid_dir, name + ".csv")
        ffd_matches.to_csv(path, index=False) #saves csv

        # Add the measurement to the faim-hcs datastructure and save.
        organoid.add_measurement(name, path)
        organoid.save() #updates json file
    
    except Exception as e:
        print(R0_obj, RX_obj, e)
        #print(R0_numpy.shape, RX_numpy.shape)
        


matching of object_12 and object_16 in day2p5 B04


100%|██████████| 49/49 [00:00<00:00, 557.52it/s]
LBFGSBOptimizerv4 (0x5576679fe9b0): LBFGSB optimizer does not support scaling. All scales are set to one.



matching of object_78 and object_92 in day2p5 B04


100%|██████████| 28/28 [00:00<00:00, 554.66it/s]


object_78 object_92 'NoneType' object has no attribute 'sum'


...

100%|██████████| 477/477 [00:01<00:00, 410.20it/s]


object_5 object_8 'NoneType' object has no attribute 'sum'


LBFGSBOptimizerv4 (0x557665e8fa30): LBFGSB optimizer does not support scaling. All scales are set to one.



matching of object_4 and object_7 in day4 B03


100%|██████████| 307/307 [00:00<00:00, 460.03it/s]
LBFGSBOptimizerv4 (0x557666041440): LBFGSB optimizer does not support scaling. All scales are set to one.



In [29]:
R0_df_org["abs_min"][0]

110.0

In [15]:
# #Do Not Run unless you know what you're doing! 
# #This is for updating the json files if writing to them got messed up

# R0.only_iterate_over_wells(False)
# R0.reset_iterator()

# for organoid in R0:
#     path = organoid.organoid_dir
#     files = listdir(path)
#     for file in files:
#         if file.startswith('linking_nuc_ffd'):
#             link_path = join(path, file)
#             name = file[:-4]
#             organoid.add_measurement(name,link_path)
#             organoid.save()
#             print(name, link_path)
            

linking_nuc_ffd_R3toR0 /tungstenfs/scratch/gliberal/Users/repinico/Microscopy/Analysis/20220528_GCPLEX_redo/20220507GCPLEX_R0/day2p5/B04/object_12/linking_nuc_ffd_R3toR0.csv
linking_nuc_ffd_R1toR0 /tungstenfs/scratch/gliberal/Users/repinico/Microscopy/Analysis/20220528_GCPLEX_redo/20220507GCPLEX_R0/day2p5/B04/object_12/linking_nuc_ffd_R1toR0.csv
linking_nuc_ffd_R2toR0 /tungstenfs/scratch/gliberal/Users/repinico/Microscopy/Analysis/20220528_GCPLEX_redo/20220507GCPLEX_R0/day2p5/B04/object_12/linking_nuc_ffd_R2toR0.csv
linking_nuc_ffd_R3toR0 /tungstenfs/scratch/gliberal/Users/repinico/Microscopy/Analysis/20220528_GCPLEX_redo/20220507GCPLEX_R0/day2p5/B04/object_78/linking_nuc_ffd_R3toR0.csv
linking_nuc_ffd_R1toR0 /tungstenfs/scratch/gliberal/Users/repinico/Microscopy/Analysis/20220528_GCPLEX_redo/20220507GCPLEX_R0/day2p5/B04/object_78/linking_nuc_ffd_R1toR0.csv
linking_nuc_ffd_R2toR0 /tungstenfs/scratch/gliberal/Users/repinico/Microscopy/Analysis/20220528_GCPLEX_redo/20220507GCPLEX_R0/day2

linking_nuc_ffd_R3toR0 /tungstenfs/scratch/gliberal/Users/repinico/Microscopy/Analysis/20220528_GCPLEX_redo/20220507GCPLEX_R0/day2p5/B04/object_14/linking_nuc_ffd_R3toR0.csv
linking_nuc_ffd_R1toR0 /tungstenfs/scratch/gliberal/Users/repinico/Microscopy/Analysis/20220528_GCPLEX_redo/20220507GCPLEX_R0/day2p5/B04/object_14/linking_nuc_ffd_R1toR0.csv
linking_nuc_ffd_R2toR0 /tungstenfs/scratch/gliberal/Users/repinico/Microscopy/Analysis/20220528_GCPLEX_redo/20220507GCPLEX_R0/day2p5/B04/object_14/linking_nuc_ffd_R2toR0.csv
linking_nuc_ffd_R1toR0 /tungstenfs/scratch/gliberal/Users/repinico/Microscopy/Analysis/20220528_GCPLEX_redo/20220507GCPLEX_R0/day2p5/B04/object_24/linking_nuc_ffd_R1toR0.csv
linking_nuc_ffd_R2toR0 /tungstenfs/scratch/gliberal/Users/repinico/Microscopy/Analysis/20220528_GCPLEX_redo/20220507GCPLEX_R0/day2p5/B04/object_24/linking_nuc_ffd_R2toR0.csv
linking_nuc_ffd_R1toR0 /tungstenfs/scratch/gliberal/Users/repinico/Microscopy/Analysis/20220528_GCPLEX_redo/20220507GCPLEX_R0/day2

linking_nuc_ffd_R3toR0 /tungstenfs/scratch/gliberal/Users/repinico/Microscopy/Analysis/20220528_GCPLEX_redo/20220507GCPLEX_R0/day2p5/B06/object_16/linking_nuc_ffd_R3toR0.csv
linking_nuc_ffd_R1toR0 /tungstenfs/scratch/gliberal/Users/repinico/Microscopy/Analysis/20220528_GCPLEX_redo/20220507GCPLEX_R0/day2p5/B06/object_16/linking_nuc_ffd_R1toR0.csv
linking_nuc_ffd_R2toR0 /tungstenfs/scratch/gliberal/Users/repinico/Microscopy/Analysis/20220528_GCPLEX_redo/20220507GCPLEX_R0/day2p5/B06/object_16/linking_nuc_ffd_R2toR0.csv
linking_nuc_ffd_R3toR0 /tungstenfs/scratch/gliberal/Users/repinico/Microscopy/Analysis/20220528_GCPLEX_redo/20220507GCPLEX_R0/day2p5/B06/object_92/linking_nuc_ffd_R3toR0.csv
linking_nuc_ffd_R1toR0 /tungstenfs/scratch/gliberal/Users/repinico/Microscopy/Analysis/20220528_GCPLEX_redo/20220507GCPLEX_R0/day2p5/B06/object_92/linking_nuc_ffd_R1toR0.csv
linking_nuc_ffd_R2toR0 /tungstenfs/scratch/gliberal/Users/repinico/Microscopy/Analysis/20220528_GCPLEX_redo/20220507GCPLEX_R0/day2

linking_nuc_ffd_R3toR0 /tungstenfs/scratch/gliberal/Users/repinico/Microscopy/Analysis/20220528_GCPLEX_redo/20220507GCPLEX_R0/day2p5/B05/object_66/linking_nuc_ffd_R3toR0.csv
linking_nuc_ffd_R1toR0 /tungstenfs/scratch/gliberal/Users/repinico/Microscopy/Analysis/20220528_GCPLEX_redo/20220507GCPLEX_R0/day2p5/B05/object_66/linking_nuc_ffd_R1toR0.csv
linking_nuc_ffd_R2toR0 /tungstenfs/scratch/gliberal/Users/repinico/Microscopy/Analysis/20220528_GCPLEX_redo/20220507GCPLEX_R0/day2p5/B05/object_66/linking_nuc_ffd_R2toR0.csv
linking_nuc_ffd_R3toR0 /tungstenfs/scratch/gliberal/Users/repinico/Microscopy/Analysis/20220528_GCPLEX_redo/20220507GCPLEX_R0/day2p5/B05/object_110/linking_nuc_ffd_R3toR0.csv
linking_nuc_ffd_R1toR0 /tungstenfs/scratch/gliberal/Users/repinico/Microscopy/Analysis/20220528_GCPLEX_redo/20220507GCPLEX_R0/day2p5/B05/object_110/linking_nuc_ffd_R1toR0.csv
linking_nuc_ffd_R2toR0 /tungstenfs/scratch/gliberal/Users/repinico/Microscopy/Analysis/20220528_GCPLEX_redo/20220507GCPLEX_R0/da

linking_nuc_ffd_R2toR0 /tungstenfs/scratch/gliberal/Users/repinico/Microscopy/Analysis/20220528_GCPLEX_redo/20220507GCPLEX_R0/day2p5/B03/object_75/linking_nuc_ffd_R2toR0.csv
linking_nuc_ffd_R3toR0 /tungstenfs/scratch/gliberal/Users/repinico/Microscopy/Analysis/20220528_GCPLEX_redo/20220507GCPLEX_R0/day2p5/B03/object_41/linking_nuc_ffd_R3toR0.csv
linking_nuc_ffd_R1toR0 /tungstenfs/scratch/gliberal/Users/repinico/Microscopy/Analysis/20220528_GCPLEX_redo/20220507GCPLEX_R0/day2p5/B03/object_41/linking_nuc_ffd_R1toR0.csv
linking_nuc_ffd_R2toR0 /tungstenfs/scratch/gliberal/Users/repinico/Microscopy/Analysis/20220528_GCPLEX_redo/20220507GCPLEX_R0/day2p5/B03/object_41/linking_nuc_ffd_R2toR0.csv
linking_nuc_ffd_R3toR0 /tungstenfs/scratch/gliberal/Users/repinico/Microscopy/Analysis/20220528_GCPLEX_redo/20220507GCPLEX_R0/day2p5/B03/object_33/linking_nuc_ffd_R3toR0.csv
linking_nuc_ffd_R1toR0 /tungstenfs/scratch/gliberal/Users/repinico/Microscopy/Analysis/20220528_GCPLEX_redo/20220507GCPLEX_R0/day2

...

linking_nuc_ffd_R3toR0 /tungstenfs/scratch/gliberal/Users/repinico/Microscopy/Analysis/20220528_GCPLEX_redo/20220507GCPLEX_R0/day3/B05/object_19/linking_nuc_ffd_R3toR0.csv
linking_nuc_ffd_R1toR0 /tungstenfs/scratch/gliberal/Users/repinico/Microscopy/Analysis/20220528_GCPLEX_redo/20220507GCPLEX_R0/day3/B05/object_19/linking_nuc_ffd_R1toR0.csv
linking_nuc_ffd_R2toR0 /tungstenfs/scratch/gliberal/Users/repinico/Microscopy/Analysis/20220528_GCPLEX_redo/20220507GCPLEX_R0/day3/B05/object_19/linking_nuc_ffd_R2toR0.csv
linking_nuc_ffd_R3toR0 /tungstenfs/scratch/gliberal/Users/repinico/Microscopy/Analysis/20220528_GCPLEX_redo/20220507GCPLEX_R0/day3/B05/object_34/linking_nuc_ffd_R3toR0.csv
linking_nuc_ffd_R1toR0 /tungstenfs/scratch/gliberal/Users/repinico/Microscopy/Analysis/20220528_GCPLEX_redo/20220507GCPLEX_R0/day3/B05/object_34/linking_nuc_ffd_R1toR0.csv
linking_nuc_ffd_R2toR0 /tungstenfs/scratch/gliberal/Users/repinico/Microscopy/Analysis/20220528_GCPLEX_redo/20220507GCPLEX_R0/day3/B05/objec

linking_nuc_ffd_R2toR0 /tungstenfs/scratch/gliberal/Users/repinico/Microscopy/Analysis/20220528_GCPLEX_redo/20220507GCPLEX_R0/day3/B06/object_16/linking_nuc_ffd_R2toR0.csv
linking_nuc_ffd_R3toR0 /tungstenfs/scratch/gliberal/Users/repinico/Microscopy/Analysis/20220528_GCPLEX_redo/20220507GCPLEX_R0/day3/B06/object_31/linking_nuc_ffd_R3toR0.csv
linking_nuc_ffd_R1toR0 /tungstenfs/scratch/gliberal/Users/repinico/Microscopy/Analysis/20220528_GCPLEX_redo/20220507GCPLEX_R0/day3/B06/object_31/linking_nuc_ffd_R1toR0.csv
linking_nuc_ffd_R2toR0 /tungstenfs/scratch/gliberal/Users/repinico/Microscopy/Analysis/20220528_GCPLEX_redo/20220507GCPLEX_R0/day3/B06/object_31/linking_nuc_ffd_R2toR0.csv
linking_nuc_ffd_R3toR0 /tungstenfs/scratch/gliberal/Users/repinico/Microscopy/Analysis/20220528_GCPLEX_redo/20220507GCPLEX_R0/day3/B06/object_71/linking_nuc_ffd_R3toR0.csv
linking_nuc_ffd_R1toR0 /tungstenfs/scratch/gliberal/Users/repinico/Microscopy/Analysis/20220528_GCPLEX_redo/20220507GCPLEX_R0/day3/B06/objec

linking_nuc_ffd_R1toR0 /tungstenfs/scratch/gliberal/Users/repinico/Microscopy/Analysis/20220528_GCPLEX_redo/20220507GCPLEX_R0/day4/B06/object_58/linking_nuc_ffd_R1toR0.csv
linking_nuc_ffd_R2toR0 /tungstenfs/scratch/gliberal/Users/repinico/Microscopy/Analysis/20220528_GCPLEX_redo/20220507GCPLEX_R0/day4/B06/object_58/linking_nuc_ffd_R2toR0.csv
linking_nuc_ffd_R3toR0 /tungstenfs/scratch/gliberal/Users/repinico/Microscopy/Analysis/20220528_GCPLEX_redo/20220507GCPLEX_R0/day4/B06/object_47/linking_nuc_ffd_R3toR0.csv
linking_nuc_ffd_R1toR0 /tungstenfs/scratch/gliberal/Users/repinico/Microscopy/Analysis/20220528_GCPLEX_redo/20220507GCPLEX_R0/day4/B06/object_47/linking_nuc_ffd_R1toR0.csv
linking_nuc_ffd_R2toR0 /tungstenfs/scratch/gliberal/Users/repinico/Microscopy/Analysis/20220528_GCPLEX_redo/20220507GCPLEX_R0/day4/B06/object_47/linking_nuc_ffd_R2toR0.csv
linking_nuc_ffd_R3toR0 /tungstenfs/scratch/gliberal/Users/repinico/Microscopy/Analysis/20220528_GCPLEX_redo/20220507GCPLEX_R0/day4/B06/objec

linking_nuc_ffd_R2toR0 /tungstenfs/scratch/gliberal/Users/repinico/Microscopy/Analysis/20220528_GCPLEX_redo/20220507GCPLEX_R0/day4/B05/object_48/linking_nuc_ffd_R2toR0.csv
linking_nuc_ffd_R3toR0 /tungstenfs/scratch/gliberal/Users/repinico/Microscopy/Analysis/20220528_GCPLEX_redo/20220507GCPLEX_R0/day4/B05/object_34/linking_nuc_ffd_R3toR0.csv
linking_nuc_ffd_R1toR0 /tungstenfs/scratch/gliberal/Users/repinico/Microscopy/Analysis/20220528_GCPLEX_redo/20220507GCPLEX_R0/day4/B05/object_34/linking_nuc_ffd_R1toR0.csv
linking_nuc_ffd_R2toR0 /tungstenfs/scratch/gliberal/Users/repinico/Microscopy/Analysis/20220528_GCPLEX_redo/20220507GCPLEX_R0/day4/B05/object_34/linking_nuc_ffd_R2toR0.csv
linking_nuc_ffd_R1toR0 /tungstenfs/scratch/gliberal/Users/repinico/Microscopy/Analysis/20220528_GCPLEX_redo/20220507GCPLEX_R0/day4/B05/object_62/linking_nuc_ffd_R1toR0.csv
linking_nuc_ffd_R3toR0 /tungstenfs/scratch/gliberal/Users/repinico/Microscopy/Analysis/20220528_GCPLEX_redo/20220507GCPLEX_R0/day4/B05/objec

linking_nuc_ffd_R2toR0 /tungstenfs/scratch/gliberal/Users/repinico/Microscopy/Analysis/20220528_GCPLEX_redo/20220507GCPLEX_R0/day4/B03/object_16/linking_nuc_ffd_R2toR0.csv
linking_nuc_ffd_R3toR0 /tungstenfs/scratch/gliberal/Users/repinico/Microscopy/Analysis/20220528_GCPLEX_redo/20220507GCPLEX_R0/day4/B03/object_31/linking_nuc_ffd_R3toR0.csv
linking_nuc_ffd_R1toR0 /tungstenfs/scratch/gliberal/Users/repinico/Microscopy/Analysis/20220528_GCPLEX_redo/20220507GCPLEX_R0/day4/B03/object_31/linking_nuc_ffd_R1toR0.csv
linking_nuc_ffd_R2toR0 /tungstenfs/scratch/gliberal/Users/repinico/Microscopy/Analysis/20220528_GCPLEX_redo/20220507GCPLEX_R0/day4/B03/object_31/linking_nuc_ffd_R2toR0.csv
linking_nuc_ffd_R3toR0 /tungstenfs/scratch/gliberal/Users/repinico/Microscopy/Analysis/20220528_GCPLEX_redo/20220507GCPLEX_R0/day4/B03/object_10/linking_nuc_ffd_R3toR0.csv
linking_nuc_ffd_R1toR0 /tungstenfs/scratch/gliberal/Users/repinico/Microscopy/Analysis/20220528_GCPLEX_redo/20220507GCPLEX_R0/day4/B03/objec

In [18]:
R0_df_ovr.loc['object_2','flag_tile_border']

False

In [19]:
R0_obj

'object_67'

In [31]:
#computed_pc_aff = affine
computed_pc_aff

array([[  1.        ,   1.        ,   0.93607054],
       [  2.        ,   2.        ,   0.95868087],
       [  3.        ,   3.        ,   0.94616995],
       [  4.        ,   4.        ,   0.97174065],
       [  5.        ,   5.        ,   0.96789523],
       [  6.        ,   9.        ,   0.93434082],
       [  7.        ,  11.        ,   0.97169236],
       [  8.        ,  12.        ,   0.96260835],
       [  9.        ,  17.        ,   0.97289598],
       [ 10.        ,  13.        ,   0.87905387],
       [ 11.        ,   6.        ,   0.88875564],
       [ 12.        ,   7.        ,   0.80691077],
       [ 13.        ,   8.        ,   0.76972909],
       [ 14.        ,  10.        ,   0.93709561],
       [ 15.        ,  14.        ,   0.97239235],
       [ 16.        ,  15.        ,   0.97372634],
       [ 17.        ,  23.        ,   0.98274489],
       [ 18.        ,  24.        ,   0.96648952],
       [ 19.        ,  16.        ,   0.97046832],
       [ 20.        ,  25.     

In [57]:
ffd

array([[  1.        ,   1.        ,   0.93607054],
       [  2.        ,   2.        ,   0.95868087],
       [  3.        ,   3.        ,   0.94616995],
       [  4.        ,   4.        ,   0.97174065],
       [  5.        ,   5.        ,   0.96789523],
       [  6.        ,   9.        ,   0.93434082],
       [  7.        ,  11.        ,   0.97169236],
       [  8.        ,  12.        ,   0.96260835],
       [  9.        ,  17.        ,   0.97289598],
       [ 10.        ,  13.        ,   0.87905387],
       [ 11.        ,   6.        ,   0.88875564],
       [ 12.        ,   7.        ,   0.80691077],
       [ 13.        ,   8.        ,   0.76972909],
       [ 14.        ,  10.        ,   0.93709561],
       [ 15.        ,  14.        ,   0.97239235],
       [ 16.        ,  15.        ,   0.97372634],
       [ 17.        ,  23.        ,   0.98274489],
       [ 18.        ,  24.        ,   0.96648952],
       [ 19.        ,  16.        ,   0.97046832],
       [ 20.        ,  25.     

In [32]:
#computed_pc_ffd = ffd
computed_pc_ffd

array([[  1.        ,   1.        ,   0.93607054],
       [  2.        ,   2.        ,   0.95868087],
       [  3.        ,   3.        ,   0.94616995],
       [  4.        ,   4.        ,   0.97174065],
       [  5.        ,   5.        ,   0.96789523],
       [  6.        ,   9.        ,   0.93434082],
       [  7.        ,  11.        ,   0.97169236],
       [  8.        ,  12.        ,   0.96260835],
       [  9.        ,  17.        ,   0.97289598],
       [ 10.        ,  13.        ,   0.87905387],
       [ 11.        ,   6.        ,   0.88875564],
       [ 12.        ,   7.        ,   0.80691077],
       [ 13.        ,   8.        ,   0.76972909],
       [ 14.        ,  10.        ,   0.93709561],
       [ 15.        ,  14.        ,   0.97239235],
       [ 16.        ,  15.        ,   0.97372634],
       [ 17.        ,  23.        ,   0.98274489],
       [ 18.        ,  24.        ,   0.96648952],
       [ 19.        ,  16.        ,   0.97046832],
       [ 20.        ,  25.     

In [51]:
R0_numpy = pd.read_csv('/tungstenfs/scratch/gliberal/Users/repinico/Microscopy/PlatyMatchTesting/PlatyMatchTest20220401-GC/org4test/round_zero.csv'
                        , sep=' ', header=None).to_numpy()  # N x 5 (first column is ids, last column is size )
RX_numpy = pd.read_csv('/tungstenfs/scratch/gliberal/Users/repinico/Microscopy/PlatyMatchTesting/PlatyMatchTest20220401-GC/org4test/round_one.csv'
                        , sep=' ', header=None).to_numpy()

In [52]:
R0_numpy

array([[1.00000000e+00, 2.23092261e+02, 1.61614853e+02, 2.16916607e+01,
        1.53370000e+04],
       [2.00000000e+00, 2.35795506e+02, 2.12385446e+02, 2.18040664e+01,
        1.96240000e+04],
       [3.00000000e+00, 1.96255551e+02, 2.46804663e+02, 2.14544148e+01,
        1.53120000e+04],
       [4.00000000e+00, 2.32972937e+02, 2.74274603e+02, 2.16585920e+01,
        1.05680000e+04],
       [5.00000000e+00, 1.79731435e+02, 3.06196383e+02, 2.17634009e+01,
        1.83010000e+04],
       [6.00000000e+00, 1.79654883e+02, 1.96500896e+02, 2.26386387e+01,
        4.01980000e+04],
       [7.00000000e+00, 1.13061212e+02, 2.11602881e+02, 2.26407395e+01,
        2.03390000e+04],
       [8.00000000e+00, 3.12650453e+02, 2.15931709e+02, 2.18678482e+01,
        1.03820000e+04],
       [9.00000000e+00, 2.96452929e+02, 2.42900919e+02, 2.20769535e+01,
        1.01230000e+04],
       [1.00000000e+01, 2.66672858e+02, 2.66145396e+02, 2.33981373e+01,
        1.87900000e+04],
       [1.10000000e+01, 1.5426

In [39]:
R0_df[['nuc_id', 'x_pos_vox', 'y_pos_vox', 'z_pos_vox', 'volume_pix']].to_numpy()
R0_numpy[:,3] *= 1/3
R0_numpy[:,4] *= 1/3
R0_numpy

array([[1.00000000e+00, 1.96255551e+02, 2.46804663e+02, 2.14544148e+01,
        1.53120000e+04],
       [2.00000000e+00, 2.02065585e+02, 2.77778783e+02, 2.23699220e+01,
        1.60250000e+04],
       [3.00000000e+00, 1.79731435e+02, 3.06196383e+02, 2.17634009e+01,
        1.83010000e+04],
       [4.00000000e+00, 2.23092261e+02, 1.61614853e+02, 2.16916607e+01,
        1.53370000e+04],
       [5.00000000e+00, 2.64333516e+02, 1.83084825e+02, 2.49012659e+01,
        2.55940000e+04],
       [6.00000000e+00, 1.79654883e+02, 1.96500896e+02, 2.26386387e+01,
        4.01980000e+04],
       [7.00000000e+00, 2.35795506e+02, 2.12385446e+02, 2.18040664e+01,
        1.96240000e+04],
       [8.00000000e+00, 1.13061212e+02, 2.11602881e+02, 2.26407395e+01,
        2.03390000e+04],
       [9.00000000e+00, 2.96452929e+02, 2.42900919e+02, 2.20769535e+01,
        1.01230000e+04],
       [1.00000000e+01, 2.66672858e+02, 2.66145396e+02, 2.33981373e+01,
        1.87900000e+04],
       [1.10000000e+01, 1.5426

In [48]:
moving_pc = R0_numpy
moving_ids = moving_pc[:, 0].transpose()  # 1 x N
moving_detections = np.flip(moving_pc[:, 1:-1], 1).transpose()  


moving_nuclei_size_mean = np.mean(moving_pc[:, -1].transpose())
moving_nuclei_size_std = np.std(moving_pc[:, -1].transpose())
moving_nuclei_size = moving_nuclei_size_mean - 3*moving_nuclei_size_std if (moving_nuclei_size_mean - 3*moving_nuclei_size_std) > 0 else (moving_nuclei_size_mean - 2*moving_nuclei_size_std)
print(moving_nuclei_size)

576.590617474747


In [24]:
    # Obtain ids
    moving_ids = moving_pc[:, 0].transpose()  # 1 x N
    moving_detections = np.flip(moving_pc[:, 1:-1], 1).transpose()  # z y x, 3 x N

    fixed_ids = fixed_pc[:, 0].transpose()
    fixed_detections = np.flip(fixed_pc[:, 1:-1], 1).transpose()

    # Calculate nucleus size
    moving_nuclei_size_mean = np.mean(moving_pc[:, -1].transpose())
    moving_nuclei_size_std = np.std(moving_pc[:, -1].transpose())
    moving_nuclei_size = moving_nuclei_size_mean - 3*moving_nuclei_size_std if (moving_nuclei_size_mean - 3*moving_nuclei_size_std) > 0 else (moving_nuclei_size_mean - 2*moving_nuclei_size_std)
    fixed_nuclei_size_mean = np.mean(fixed_pc[:, -1].transpose())
    fixed_nuclei_size_std = np.std(fixed_pc[:, -1].transpose())
    fixed_nuclei_size = fixed_nuclei_size_mean - 3*fixed_nuclei_size_std if (fixed_nuclei_size_mean - 3*fixed_nuclei_size_std) > 0 else (fixed_nuclei_size_mean - 2*fixed_nuclei_size_std)

    ransac_error = 0.5 * (moving_nuclei_size ** (1 / 3) + fixed_nuclei_size ** (1 / 3))
    print("=" * 25)
    print("Calculated approximate nucleus radius equal to {:.3f}".format(ransac_error))

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 

In [19]:
R0_seg

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 

In [4]:
R0.reset_iterator()

for organoid in R0:
#     affine_matches = None
#     ffd_matches = None
    
    well_id = organoid.well.well_id
    plate_id = organoid.well.plate.plate_id
    
    #load R0 data
    R0_obj = organoid.organoid_id
    R0_id = int(R0_obj.rpartition("_")[2])
    
    if R0_id != 4: #move to next organoid if no match
        continue
       
    R0_df = organoid.get_measurement('regionprops_nuc_C01')
    R0_raw = organoid.get_raw_data(ovr_channel)
    R0_seg = organoid.get_segmentation('NUC_SEG3D_20210828_finetune')
    
    #load linking data
    link_org = organoid.well.get_measurement('linking_ovr_C01_R1toR0')
    link_org_dict = link_org.set_index('R0_label').T.to_dict('index')['RX_label'] #R0 is key, R1 is value
    
    #load RX data
    if R0_id not in link_org_dict: #move to next organoid if no match
        continue
       
    RX_id = link_org_dict[R0_id]
    RX_obj = "object_"+ str(RX_id)
    
    RX_df = RX.plates[plate_id].wells[well_id].organoids[RX_obj].get_measurement('regionprops_nuc_C01')
    RX_raw = RX.plates[plate_id].wells[well_id].organoids[RX_obj].get_raw_data(ovr_channel)
    RX_seg = RX.plates[plate_id].wells[well_id].organoids[RX_obj].get_segmentation('NUC_SEG3D_20210828_finetune')
    
    #convert df's to PlatyMatch compatible numpy array
    # N x 5 (first column is ids, last column is size)
    R0_numpy = R0_df[['nuc_id', 'x_pos_vox', 'y_pos_vox', 'z_pos_vox', 'volume_pix']].to_numpy()
    RX_numpy = RX_df[['nuc_id', 'x_pos_vox', 'y_pos_vox', 'z_pos_vox', 'volume_pix']].to_numpy()
    
    R0_numpy = pd.read_csv('/tungstenfs/scratch/gliberal/Users/repinico/Microscopy/PlatyMatchTesting/PlatyMatchTest20220401-GC/220314EMS005_O00004/round_zero.csv'
                        , sep=' ', header=None).to_numpy()  # N x 5 (first column is ids, last column is size )
    RX_numpy = pd.read_csv('/tungstenfs/scratch/gliberal/Users/repinico/Microscopy/PlatyMatchTesting/PlatyMatchTest20220401-GC/220314EMS005_O00004/round_one.csv'
                        , sep=' ', header=None).to_numpy()
    
    # Load Raw Images
    R0_raw = tifffile.imread('/tungstenfs/scratch/gliberal/Users/repinico/Microscopy/PlatyMatchTesting/PlatyMatchTest20220401-GC/220314EMS005_O00004/R0_4_raw.tif')
    R1_raw = tifffile.imread('/tungstenfs/scratch/gliberal/Users/repinico/Microscopy/PlatyMatchTesting/PlatyMatchTest20220401-GC/220314EMS005_O00004/R1_4_raw.tif')

    # Load Labels
    R0_seg = tifffile.imread('/tungstenfs/scratch/gliberal/Users/repinico/Microscopy/PlatyMatchTesting/PlatyMatchTest20220401-GC/220314EMS005_O00004/R0_4_pred.tif')
    R1_seg = tifffile.imread('/tungstenfs/scratch/gliberal/Users/repinico/Microscopy/PlatyMatchTesting/PlatyMatchTest20220401-GC/220314EMS005_O00004/R1_4_pred.tif')

    
    #skip organoids that have less than 4 nuclei
    if (R0_numpy.shape[0] <= 4) | (RX_numpy.shape[0] <= 4): #move to next organoid if no match
        continue
        
    #run PlatyMatch
    warnings.filterwarnings("ignore")
    ransac_iterations=4000
    icp_iterations=50
    
    print("affine matching of", R0_obj, "and", RX_obj)
    # Perform affine matching
    try:
#         row_ids, col_ids, transform_matrix_combined = register_affine(RX_numpy, R0_numpy, ransac_iterations, icp_iterations)

#         affine_matches = pd.DataFrame(np.stack([col_ids, row_ids]).T.astype(np.int16), columns=['R0_nuc_id', 'RX_nuc_id'])
        affine_matches, transform_matrix_combined, confidence = register_affine(RX_numpy, R0_numpy, ransac_iterations, icp_iterations)
        print(confidence)
        affine_matches = pd.DataFrame(affine_matches, columns=['R0_nuc_id', 'RX_nuc_id', 'confidence'])
        affine_matches['R0_organoid_id'] = R0_obj
        affine_matches['RX_organoid_id'] = RX_obj
        affine_matches['plate_id'] = plate_id
        affine_matches['well_id'] = well_id
        
        # Save measurement into the organoid directory.
        name = "linking_nuc_affine_terminal_" + names[1] + "to" + names[0]
        path = join(organoid.organoid_dir, name + ".csv")
        affine_matches.to_csv(path, index=False) #saves csv

        # Add the measurement to the faim-hcs datastructure and save.
        organoid.add_measurement(name, path)
        organoid.save() #updates json file
    
    except Exception as e:
        print(R0_obj, RX_obj, e)
        #print(R0_numpy.shape, RX_numpy.shape)
    
    print("ffd matching of", R0_obj, "and", RX_obj)
    # Perform FFD matching
    try:
#         ffd_row_ids, ffd_col_ids = register_ffd(RX_numpy, R0_numpy, transform_matrix_combined, RX_raw, R0_raw, RX_seg, R0_seg)
        
#         ffd_matches = pd.DataFrame(np.stack([ffd_col_ids, ffd_row_ids]).T.astype(np.int16), columns=['R0_nuc_id', 'RX_nuc_id'])
        ffd_matches = register_ffd(RX_numpy, R0_numpy, transform_matrix_combined, RX_raw, R0_raw, RX_seg, R0_seg)
        ffd_matches = pd.DataFrame(ffd_matches, columns=['R0_nuc_id', 'RX_nuc_id'])
        ffd_matches['R0_organoid_id'] = R0_obj
        ffd_matches['RX_organoid_id'] = RX_obj
        ffd_matches['plate_id'] = plate_id
        ffd_matches['well_id'] = well_id
        
        # Save measurement into the organoid directory.
        name = "linking_nuc_ffd_terminal_" + names[1] + "to" + names[0]
        path = join(organoid.organoid_dir, name + ".csv")
        ffd_matches.to_csv(path, index=False) #saves csv

        # Add the measurement to the faim-hcs datastructure and save.
        organoid.add_measurement(name, path)
        organoid.save() #updates json file
    
    except Exception as e:
        print(R0_obj, RX_obj, e)





affine matching of object_4 and object_4


100%|██████████| 170/170 [00:14<00:00, 11.36it/s]


[[0.93607054 0.18815324 0.26277767 ... 0.18459525 0.2974667  0.17657349]
 [0.16584462 0.95868087 0.23339092 ... 0.20707565 0.1875008  0.28789471]
 [0.24827649 0.27972815 0.94616995 ... 0.13607095 0.555271   0.20873802]
 ...
 [0.18052329 0.20021445 0.10973824 ... 0.6333336  0.14052264 0.19259406]
 [0.23491425 0.26049144 0.22928993 ... 0.22397922 0.23267997 0.45579765]
 [0.15602648 0.3373263  0.22306242 ... 0.17632048 0.46459508 0.95556267]]
ffd matching of object_4 and object_4


LBFGSBOptimizerv4 (0x557619f73a60): LBFGSB optimizer does not support scaling. All scales are set to one.



In [8]:
ffd_matches

,R0_nuc_id,RX_nuc_id,confidence,R0_organoid_id,RX_organoid_id,plate_id,well_id
0,1.0,1.0,0.936071,object_4,object_4,day3,E02
1,2.0,2.0,0.958681,object_4,object_4,day3,E02
2,3.0,3.0,0.946170,object_4,object_4,day3,E02
3,4.0,4.0,0.971741,object_4,object_4,day3,E02
4,5.0,5.0,0.967895,object_4,object_4,day3,E02
5,6.0,6.0,0.203491,object_4,object_4,day3,E02
6,7.0,7.0,0.510217,object_4,object_4,day3,E02
7,8.0,8.0,0.114786,object_4,object_4,day3,E02
8,9.0,9.0,0.329485,object_4,object_4,day3,E02
9,10.0,10.0,0.191296,object_4,object_4,day3,E02


In [ ]:
'/tungstenfs/scratch/gliberal/Users/repinico/Microscopy/PlatyMatchTesting/PlatyMatchTest20220401-GC/220314EMS005_O00004/round_zero.csv'

In [11]:
moving_pc = pd.read_csv('/tungstenfs/scratch/gliberal/Users/repinico/Microscopy/PlatyMatchTesting/PlatyMatchTest20220401-GC/220314EMS005_O00004/round_one.csv'
                        , sep=' ', header=None).to_numpy()  # N x 5 (first column is ids, last column is size )
fixed_pc = pd.read_csv('/tungstenfs/scratch/gliberal/Users/repinico/Microscopy/PlatyMatchTesting/PlatyMatchTest20220401-GC/220314EMS005_O00004/round_zero.csv'
                        , sep=' ', header=None).to_numpy()


In [12]:
fixed_pc

array([[1.00000000e+00, 2.14699425e+02, 1.55008465e+02, 1.87973679e+01,
        1.51210000e+04],
       [2.00000000e+00, 2.27665379e+02, 2.05313150e+02, 1.87511070e+01,
        1.94220000e+04],
       [3.00000000e+00, 1.88568347e+02, 2.38772110e+02, 1.84528226e+01,
        1.48800000e+04],
       [4.00000000e+00, 2.24611645e+02, 2.66418990e+02, 1.87496908e+01,
        1.05110000e+04],
       [5.00000000e+00, 1.72892094e+02, 2.96742175e+02, 1.87439105e+01,
        1.67090000e+04],
       [6.00000000e+00, 2.87083981e+02, 2.35334114e+02, 1.93083555e+01,
        9.39500000e+03],
       [7.00000000e+00, 1.47663726e+02, 2.61488247e+02, 1.95104126e+01,
        1.79110000e+04],
       [8.00000000e+00, 1.94190976e+02, 2.69815506e+02, 1.92590432e+01,
        1.54260000e+04],
       [9.00000000e+00, 1.77943515e+02, 1.19094162e+02, 2.19069980e+01,
        1.63760000e+04],
       [1.00000000e+01, 7.24149333e+01, 1.56293071e+02, 1.95651270e+01,
        1.39420000e+04],
       [1.10000000e+01, 1.7267